In [ ]:
%%capture
%config Completer.use_jedi = False
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

# Install on Google Colab
import subprocess
import sys

from IPython import get_ipython

install_packages = "google.colab" in str(get_ipython())
if install_packages:
    for package in ["qrules[doc]", "graphviz"]:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", package]
        )

# Visualize decay topologies

````{margin}
```{warning}
{class}`graphviz.Source` requires your system to have DOT installed, see {doc}`Installation <graphviz:index>`.
```
````

The {mod}`~qrules.io` module allows you to convert {class}`.StateTransitionGraph`, {class}`.Topology` instances, and {class}`.ProblemSet`s to [DOT language](https://graphviz.org/doc/info/lang.html) with {func}`.asdot`. You can visualize its output with third-party libraries, such as [Graphviz](https://graphviz.org). This is particularly useful after running {meth}`~.StateTransitionManager.find_solutions`, which produces a {class}`.ReactionInfo` object with a {class}`.list` of {class}`.StateTransitionGraph` instances (see {doc}`/usage/reaction`).

## Topologies

First of all, here are is an example of how to visualize a group of {class}`.Topology` instances. We use {func}`.create_isobar_topologies` and {func}`.create_n_body_topology` to create a few standard topologies.

In [ ]:
import graphviz

import qrules
from qrules.topology import create_isobar_topologies, create_n_body_topology

In [ ]:
topology = create_n_body_topology(2, 4)
graphviz.Source(qrules.io.asdot(topology, render_initial_state_id=True))

Note the IDs of the {attr}`~.Topology.nodes` is also rendered if there is more than node:

In [ ]:
topologies = create_isobar_topologies(4)
graphviz.Source(qrules.io.asdot(topologies))

This can be turned on or off with the arguments of {func}`.asdot`:

In [ ]:
topologies = create_isobar_topologies(3)
graphviz.Source(qrules.io.asdot(topologies, render_node=False))

{func}`.asdot` provides other options as well:

In [ ]:
topologies = create_isobar_topologies(5)
dot = qrules.io.asdot(
    topologies[0],
    render_final_state_id=False,
    render_resonance_id=True,
    render_node=False,
)
display(graphviz.Source(dot))

## {class}`.ProblemSet`s

As noted in {doc}`reaction`, the {class}`.StateTransitionManager` provides more control than the façade function {func}`.generate_transitions`. One advantages, is that the {class}`.StateTransitionManager` first generates a set of {class}`.ProblemSet`s with {meth}`.create_problem_sets` that you can further configure if you wish.

In [ ]:
stm = qrules.StateTransitionManager(
    initial_state=["J/psi(1S)"],
    final_state=["K0", "Sigma+", "p~"],
    formalism="canonical-helicity",
)
problem_sets = stm.create_problem_sets()

Note that the output of {meth}`.create_problem_sets` is a {obj}`dict` with {obj}`float` values as keys (representing the interaction strength) and {obj}`list`s of {obj}`.ProblemSet`s as values.

In [ ]:
sorted(problem_sets, reverse=True)

In [ ]:
len(problem_sets[60.0])

In [ ]:
problem_set = problem_sets[60.0][0]
dot = qrules.io.asdot(problem_set, render_node=True)
graphviz.Source(dot)

## {class}`.StateTransition`s

Here, we'll visualize the allowed transitions for the decay $\psi' \to \gamma\eta\eta$ as an example.

In [ ]:
import qrules

reaction = qrules.generate_transitions(
    initial_state="psi(2S)",
    final_state=["gamma", "eta", "eta"],
    allowed_interaction_types="EM",
)

As noted in {ref}`usage/reaction:3. Find solutions`, the {attr}`~.ReactionInfo.transitions` contain all spin projection combinations (which is necessary for the {mod}`ampform` package). It is possible to convert all these solutions to DOT language with {func}`~.asdot`. To avoid visualizing all solutions, we just take a subset of the {attr}`~.ReactionInfo.transitions`:

In [ ]:
dot = qrules.io.asdot(reaction.transitions[::50][:3])  # just some selection

This {class}`str` of [DOT language](https://graphviz.org/doc/info/lang.html) for the list of {class}`.StateTransitionGraph` instances can then be visualized with a third-party library, for instance, with {class}`graphviz.Source`:

In [ ]:
import graphviz

dot = qrules.io.asdot(
    reaction.transitions[::50][:3], render_node=False
)  # just some selection
graphviz.Source(dot)

You can also serialize the DOT string to file with {func}`.io.write`. The file extension for a DOT file is `.gv`:

In [ ]:
qrules.io.write(reaction, "decay_topologies_with_spin.gv")

### Collapse graphs

Since this list of all possible spin projections {attr}`~.ReactionInfo.transitions` is rather long, it is often useful to use `strip_spin=True` or `collapse_graphs=True` to bundle comparable graphs. First, {code}`strip_spin=True` allows one collapse (ignore) the spin projections (we again show a selection only):

In [ ]:
dot = qrules.io.asdot(reaction.transitions[:3], strip_spin=True)
graphviz.Source(dot)

```{note}
By default, `.asdot` renders edge IDs, because they represent the (final) state IDs as well. In the example above, we switched this off.
```

If that list is still too much, there is {code}`collapse_graphs=True`, which bundles all graphs with the same final state groupings:

In [ ]:
dot = qrules.io.asdot(reaction, collapse_graphs=True, render_node=False)
graphviz.Source(dot)